In [2]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from dataclasses import dataclass
from scipy.spatial.transform import Rotation as R  # rotation axis ??? left-hand / clockwise
from typing import List, Tuple
from pathlib import Path as path

%matplotlib ipympl

In [14]:
'''
    Get 3D label and bounding boxes (from two images) for each scene
'''
def get_location(captures):
    # coeff = np.array([-0.00389254, 0.49512566, 0.13212298])

    def _find(l, s):
        for elem in l:
            if elem['id'] == s or elem['id'] == f'{s}_0':
                values = elem['values']
                for i, value in enumerate(values):
                    if value['labelName'] == 'human':
                        break
                return values[i]
    
    def _get_bbox(view):
        '''
            For each view, extract the 2D bounding box and
        '''
        bbox_3d, bbox_2d = _find(view['annotations'], 'bounding box 3D'), _find(view['annotations'], 'bounding box')
        
        # refine 3D location of the object
        euler = R.from_quat(bbox_3d.get('rotation')).as_matrix()
        obj_location = view.get('position') + np.dot(bbox_3d['translation'], euler) # - bbox_3d['size']*np.array([-0.00389254, 0.49512566, 0.13212298])
        # pre-process 2D bbox
        bbox = {
            'center': np.array(bbox_2d['origin']) + np.array(bbox_2d['dimension'])/2,
            'size': np.array(bbox_2d['dimension'])/2,
        }

        return obj_location, bbox

    angles = [
        [10, 245, 0],
        [7, 120, 0],
    ]
    bboxes, cameras = [], []
    for i, view in enumerate(captures):  # per view
        obj_location, bbox_2d = _get_bbox(view)

        bboxes.append(bbox_2d)
        cameras.append({
            'filename': view['filename'],
            'position': view['position'],
            'rotation': angles[i],
        })

    return {
        '3D_location': obj_location,
        'bboxes': bboxes,
        'cameras': cameras,
    }


In [10]:
@dataclass
class Camera:
    position: np.ndarray#=field(default_factory=np.array([0., 0., 0.]))
    # angle: np.ndarray#=field(default_factory=np.array([0., 1., 0.]))  # angle in degrees
    angle: np.ndarray#=field(default_factory=np.array([0., 1., 0.]))  # angle in quaternion

    focal_length: float=20.
    resolution: np.ndarray=np.array([3840, 2160])
    sensor_size: np.ndarray=np.array([30, 30])
    fov: float=73.73979

    def __post_init__(self):
        print('info', self.position, self.angle)
        pitch, yaw, roll = self.angle  # R_x, R_y, R_z

        sin_yaw, cos_yaw = np.sin(np.deg2rad(yaw)), np.cos(np.deg2rad(yaw))
        sin_pitch, cos_pitch = np.sin(np.deg2rad(pitch)), np.cos(np.deg2rad(pitch))
        sin_roll, cos_roll = np.sin(np.deg2rad(roll)), np.cos(np.deg2rad(roll))
        
        self.rotation_yaw = np.array([
            [cos_yaw, 0, -sin_yaw],
            [0, 1, 0],
            [sin_yaw, 0, cos_yaw],       
        ])
        self.rotation_pitch = np.array([
            [1, 0, 0],
            [0, cos_pitch, -sin_pitch],
            [0, sin_pitch, cos_pitch],
        ])
        self.rotation_roll = np.array([
            [cos_roll, -sin_roll, 0],
            [sin_roll, cos_roll, 0],
            [0, 0, 1]
        ])
        self.rotation = self.rotation_yaw @ self.rotation_pitch @ self.rotation_roll

        self.intrinsic = np.array([
            [self.focal_length*self.resolution[0]/self.sensor_size[0], 0, self.resolution[0]/2],
            [0, self.focal_length*self.resolution[1]/self.sensor_size[1], self.resolution[1]/2],
            [0, 0, 1]
        ])

    def pixel2ray(self, pixel):
        pixel = np.append(pixel, np.zeros((pixel.shape[0], 1)) + 1, axis=-1)
        camera_coor = pixel @ np.linalg.inv(self.intrinsic)
        world_coor = (camera_coor @ self.rotation)

        vector = world_coor - np.array(self.position)
        directional_vector = vector / np.linalg.norm(vector)
        
        return [Line(origin=coor, direction=vec) for coor, vec in zip(world_coor, directional_vector)]


@dataclass
class Line:
    origin: np.ndarray
    direction: np.ndarray


def find_points(line_a: Line, line_b: Line):
    n = np.cross(line_a.direction, line_b.direction)
    d = np.abs(np.dot(n, line_a.origin - line_b.origin)) / np.linalg.norm(n)
    
    t_a = np.dot(np.cross(line_b.direction, n), (line_b.origin - line_a.origin)) / np.dot(n, n)
    t_b = np.dot(np.cross(line_a.direction, n), (line_b.origin - line_a.origin)) / np.dot(n, n)

    p_a = line_a.origin + t_a * line_a.direction
    p_b = line_b.origin + t_b * line_b.direction

    return (p_a + p_b) / 2

colors = ['red', 'blue']

In [15]:
folder_path = path.joinpath(path.cwd(), 'solo')

for i in range(10):  # per scene
    # Extract 3D label and bboxes of each image from a scene
    file_path = path.joinpath(folder_path, f'sequence.{i}')
    with open(path.joinpath(file_path, 'step0.frame_data.json')) as json_file:
        data = json.load(json_file)
    captures = data['captures']
    output = get_location(captures)
    bboxes, cameras = output['bboxes'], output['cameras']

    print(bboxes)
    print(cameras)

    cameras_ = [Camera(position=camera['position'], angle=camera['rotation']) for camera in cameras]

    lines = []
    for i, (bbox, camera) in enumerate(zip(output.get('bboxes'), cameras_)):
        pixels = np.array([
            bbox['center']
        ])
        rays = camera.pixel2ray(pixels)
        stack_origin = np.stack([camera.position]*pixels.shape[0])
        lines.append(rays[-1])

    point = find_points(*lines)
    print('lpm', np.sum((point - output['3D_location'])**2))
    print(point)
    print(output['3D_location'])

    break


[0.046828758, -0.8401821, 0.0735064447, -0.535254955]


ValueError: too many values to unpack (expected 3)